In [1]:
from audiomentations import AddBackgroundNoise, PolarityInversion
from imutils import paths
import soundfile as sf
import numpy as np
import librosa
import random
import shutil
import json
import os 

In [60]:
# path to the synthetic data
dataset_path = "synthetic_tts_dataset/"

# path to save the processed dataset
save_dataset_path = "Keyword-MLP/data_issai/"

# path to the background noise dataset
bn_dataset_path = "ESC-50-master/audio" 

n_augs = 5

commands = ["issai", "other"]

In [23]:
# create new directories
for command in commands:
    command_path = os.path.join(save_dataset_path, command)
    if not os.path.exists(command_path):
        os.makedirs(command_path)

In [39]:
# check the num of speech commands in KSCD
wavPaths = list(paths.list_files(dataset_path, validExts="wav"))
print(len(wavPaths))

21087


In [20]:
# https://github.com/musikalkemist/audioDataAugmentationTutorial/blob/main/3/dataaugmentation.py
def add_white_noise(signal, noise_percentage_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_percentage_factor
    return augmented_signal


def time_stretch(signal, time_stretch_rate):
    return librosa.effects.time_stretch(y=signal, rate=time_stretch_rate)


def pitch_scale(signal, sample_rate, num_semitones):
    return librosa.effects.pitch_shift(y=signal, sr=sample_rate, n_steps=num_semitones)


def random_gain(signal, min_factor, max_factor):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal


def invert_polarity(signal):
    return signal * -1


add_background_noise = AddBackgroundNoise(sounds_path=bn_dataset_path,
                                          min_snr_in_db=5.0,
                                          max_snr_in_db=30.0,
                                          noise_transform=PolarityInversion(),
                                          p=1.0)

In [63]:
def helper(wavPath):
    command, wav_file = wavPath.split('/')[-2:]
    
    wav_file_name = wav_file.split('.')[0]
    
    model, speaker, *remain = wav_file_name.split('_')
        
    if int(speaker) < 750:
        set_ = "train"
    elif int(speaker) >= 750 and int(speaker) < 800:
        set_ = "val"
    else:
        set_ = "test"
        
    return set_, wav_file_name, command

In [67]:
# create training, validation, and test lists
train_list = []
val_list = []
test_list = []
sample_rate = 16000

# loop over the speech commands
for i, wavPath in enumerate(wavPaths):
    print("[INFO] Processing file: {}/{}".format(i+1, len(wavPaths)))
    raw_signal, _ = librosa.load(wavPath, sr=sample_rate)
    set_, wav_file_name, command = helper(wavPath)
    save_path = os.path.join(save_dataset_path, command)

    sf.write(os.path.join(save_path, "{}.wav".format(wav_file_name)), raw_signal, sample_rate)
    if set_ == "train":
        train_list.append('./data_issai/' + command + '/' + "{}.wav".format(wav_file_name))
    elif set_ == "val":
        val_list.append('./data_issai/' + command + '/' + "{}.wav".format(wav_file_name))
    else:
        test_list.append('./data_issai/' + command + '/' + "{}.wav".format(wav_file_name))

    for i in range(n_augs):
        bn_signal = add_background_noise(raw_signal, sample_rate=sample_rate)
        bn_wav_file_name = "{}-{}.wav".format(wav_file_name, i)
        sf.write(os.path.join(save_path, bn_wav_file_name), bn_signal, sample_rate)
        
        if set_ == "train":
            train_list.append('./data_issai/' + command + '/' + bn_wav_file_name)
        elif set_ == "val":
            val_list.append('./data_issai/' + command + '/' + bn_wav_file_name)
        else:
            test_list.append('./data_issai/' + command + '/' + bn_wav_file_name)

[INFO] Processing file: 1/21087
[INFO] Processing file: 2/21087
[INFO] Processing file: 3/21087
[INFO] Processing file: 4/21087
[INFO] Processing file: 5/21087
[INFO] Processing file: 6/21087
[INFO] Processing file: 7/21087
[INFO] Processing file: 8/21087


/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/3-163459-A-0.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/5-177614-A-5.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 9/21087
[INFO] Processing file: 10/21087
[INFO] Processing file: 11/21087
[INFO] Processing file: 12/21087
[INFO] Processing file: 13/21087
[INFO] Processing file: 14/21087
[INFO] Processing file: 15/21087


/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/5-179866-A-43.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/4-197871-A-15.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/3-145577-A-43.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(
/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_

[INFO] Processing file: 16/21087
[INFO] Processing file: 17/21087
[INFO] Processing file: 18/21087
[INFO] Processing file: 19/21087
[INFO] Processing file: 20/21087
[INFO] Processing file: 21/21087
[INFO] Processing file: 22/21087
[INFO] Processing file: 23/21087


...

[INFO] Processing file: 15347/21087
[INFO] Processing file: 15348/21087
[INFO] Processing file: 15349/21087
[INFO] Processing file: 15350/21087
[INFO] Processing file: 15351/21087
[INFO] Processing file: 15352/21087
[INFO] Processing file: 15353/21087
[INFO] Processing file: 15354/21087
[INFO] Processing file: 15355/21087
[INFO] Processing file: 15356/21087
[INFO] Processing file: 15357/21087
[INFO] Processing file: 15358/21087
[INFO] Processing file: 15359/21087
[INFO] Processing file: 15360/21087
[INFO] Processing file: 15361/21087
[INFO] Processing file: 15362/21087
[INFO] Processing file: 15363/21087
[INFO] Processing file: 15364/21087
[INFO] Processing file: 15365/21087
[INFO] Processing file: 15366/21087
[INFO] Processing file: 15367/21087
[INFO] Processing file: 15368/21087
[INFO] Processing file: 15369/21087
[INFO] Processing file: 15370/21087
[INFO] Processing file: 15371/21087
[INFO] Processing file: 15372/21087
[INFO] Processing file: 15373/21087
[INFO] Processing file: 1537

/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/2-43806-A-42.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 16952/21087
[INFO] Processing file: 16953/21087
[INFO] Processing file: 16954/21087
[INFO] Processing file: 16955/21087
[INFO] Processing file: 16956/21087
[INFO] Processing file: 16957/21087
[INFO] Processing file: 16958/21087
[INFO] Processing file: 16959/21087
[INFO] Processing file: 16960/21087
[INFO] Processing file: 16961/21087
[INFO] Processing file: 16962/21087
[INFO] Processing file: 16963/21087
[INFO] Processing file: 16964/21087
[INFO] Processing file: 16965/21087
[INFO] Processing file: 16966/21087
[INFO] Processing file: 16967/21087
[INFO] Processing file: 16968/21087
[INFO] Processing file: 16969/21087
[INFO] Processing file: 16970/21087
[INFO] Processing file: 16971/21087
[INFO] Processing file: 16972/21087
[INFO] Processing file: 16973/21087
[INFO] Processing file: 16974/21087
[INFO] Processing file: 16975/21087
[INFO] Processing file: 16976/21087
[INFO] Processing file: 16977/21087
[INFO] Processing file: 16978/21087
[INFO] Processing file: 1697

/home/admin2/.local/lib/python3.8/site-packages/audiomentations/augmentations/add_background_noise.py:128: UserWarning: The file /home/admin2/askat_kuzdeuov/kscd/ESC-50-master/audio/5-103421-A-2.wav is too silent to be added as noise. Returning the input unchanged.
  warnings.warn(


[INFO] Processing file: 18478/21087
[INFO] Processing file: 18479/21087
[INFO] Processing file: 18480/21087
[INFO] Processing file: 18481/21087
[INFO] Processing file: 18482/21087
[INFO] Processing file: 18483/21087
[INFO] Processing file: 18484/21087
[INFO] Processing file: 18485/21087
[INFO] Processing file: 18486/21087
[INFO] Processing file: 18487/21087
[INFO] Processing file: 18488/21087
[INFO] Processing file: 18489/21087
[INFO] Processing file: 18490/21087
[INFO] Processing file: 18491/21087
[INFO] Processing file: 18492/21087
[INFO] Processing file: 18493/21087
[INFO] Processing file: 18494/21087
[INFO] Processing file: 18495/21087
[INFO] Processing file: 18496/21087
[INFO] Processing file: 18497/21087
[INFO] Processing file: 18498/21087
[INFO] Processing file: 18499/21087
[INFO] Processing file: 18500/21087
[INFO] Processing file: 18501/21087
[INFO] Processing file: 18502/21087
[INFO] Processing file: 18503/21087
[INFO] Processing file: 18504/21087
[INFO] Processing file: 1850

In [79]:
# check the num of speech commands in KSCD
bnwavPaths = list(paths.list_files(bn_dataset_path, validExts="wav"))
print(len(bnwavPaths))

2000


In [83]:
# loop over the speech commands
for i, wavPath in enumerate(bnwavPaths):
    print("[INFO] Processing file: {}/{}".format(i+1, len(bnwavPaths)))
    raw_signal, _ = librosa.load(wavPath, sr=sample_rate)
    wav_file_name = wavPath.split('/')[-1]
    save_path = os.path.join(save_dataset_path, "other")
    
    sf.write(os.path.join(save_path, wav_file_name), raw_signal, sample_rate)
    
    if i < 1700:
        train_list.append('./data_issai/' + "other" + '/' + wav_file_name)
    elif i >= 1700 and i < 1800:
        val_list.append('./data_issai/' + "other" + '/' + wav_file_name)
    else:
        test_list.append('./data_issai/' + "other" + '/' + wav_file_name)

[INFO] Processing file: 1/2000
[INFO] Processing file: 2/2000
[INFO] Processing file: 3/2000
[INFO] Processing file: 4/2000
[INFO] Processing file: 5/2000
[INFO] Processing file: 6/2000
[INFO] Processing file: 7/2000
[INFO] Processing file: 8/2000
[INFO] Processing file: 9/2000
[INFO] Processing file: 10/2000
[INFO] Processing file: 11/2000
[INFO] Processing file: 12/2000
[INFO] Processing file: 13/2000
[INFO] Processing file: 14/2000
[INFO] Processing file: 15/2000
[INFO] Processing file: 16/2000
[INFO] Processing file: 17/2000
[INFO] Processing file: 18/2000
[INFO] Processing file: 19/2000
[INFO] Processing file: 20/2000
[INFO] Processing file: 21/2000
[INFO] Processing file: 22/2000
[INFO] Processing file: 23/2000
[INFO] Processing file: 24/2000
[INFO] Processing file: 25/2000
[INFO] Processing file: 26/2000
[INFO] Processing file: 27/2000
[INFO] Processing file: 28/2000
[INFO] Processing file: 29/2000
[INFO] Processing file: 30/2000
[INFO] Processing file: 31/2000
[INFO] Processing

In [89]:
label_list = [label for label in sorted(os.listdir(save_dataset_path)) if os.path.isdir(os.path.join(save_dataset_path, label)) and label[0] != "_"]
label_map = {idx: label for idx, label in enumerate(label_list)}

with open(os.path.join(save_dataset_path, "training_list.txt"), "w+") as f:
    f.write("\n".join(train_list))

with open(os.path.join(save_dataset_path, "validation_list.txt"), "w+") as f:
    f.write("\n".join(val_list))

with open(os.path.join(save_dataset_path, "testing_list.txt"), "w+") as f:
    f.write("\n".join(test_list))

with open(os.path.join(save_dataset_path, "label_map.json"), "w+") as f:
    json.dump(label_map, f)

In [93]:
print("Train samples:", len(train_list))
print("Val samples:", len(val_list))
print("Test samples:", len(test_list))

Train samples: 107894
Val samples: 6700
Test samples: 13928
